In [1]:
apikey = 'QddE1UndJCJ48XvSgC6xKZV2tR365sqFCvpdX6mT6xHOcB9a7Ykqu30qyNn8znOe'
secret = 'kyArU6V1ejdtrZHFpk5CmLpJG1Lk4inAMnoC8hgX53RtK3zDjdUOwV6VH63d6a5B'

In [4]:
#!pip install python-binance pandas mplfinance

In [5]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd

In [6]:
client = Client(apikey, secret)

In [7]:
tickers = client.get_all_tickers()

In [12]:
depth = client.get_order_book(symbol = 'BTCUSDT')

In [16]:
depth.keys()

dict_keys(['lastUpdateId', 'bids', 'asks'])

In [21]:
bids = pd.DataFrame(depth['bids'])
bids.columns = ['b_price', 'b_vol']
bids.head(2)

,b_price,b_vol
0,19893.13000000,0.98897000
1,19892.97000000,0.00100000


In [22]:
asks = pd.DataFrame(depth['asks'])
asks.columns = ['a_price', 'a_vol']
asks.head(2)

,a_price,a_vol
0,19893.14000000,4.81294000
1,19893.19000000,0.12573000


In [30]:
data = pd.concat([bids, asks], axis = 1, join = 'inner')

In [31]:
data

,b_price,b_vol,a_price,a_vol
0,19893.13000000,0.98897000,19893.14000000,4.81294000
1,19892.97000000,0.00100000,19893.19000000,0.12573000
2,19891.48000000,0.04122000,19893.93000000,0.00100000
3,19891.47000000,0.12404000,19894.11000000,0.56127000
4,19890.22000000,0.00100000,19894.12000000,1.99061000
...,...,...,...,...
95,19878.64000000,0.00210000,19914.00000000,0.12582000
96,19878.37000000,0.10000000,19914.91000000,0.00276000
97,19878.35000000,0.22000000,19915.13000000,0.00916000
98,19877.94000000,0.00310000,19915.15000000,0.00057000


In [32]:
hist = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_1HOUR, '1 Jan 2018')

In [ ]:
#  [
#   [
#     1499040000000,      // Open time
#     "0.01634790",       // Open
#     "0.80000000",       // High
#     "0.01575800",       // Low
#     "0.01577100",       // Close
#     "148976.11427815",  // Volume
#     1499644799999,      // Close time
#     "2434.19055334",    // Quote asset volume
#     308,                // Number of trades
#     "1756.87402397",    // Taker buy base asset volume
#     "28.46694368",      // Taker buy quote asset volume
#     "17928899.62484339" // Ignore.
#   ]
# ]

In [37]:
hist[0][0]

1514764800000

In [38]:
hist_df = pd.DataFrame(hist)

In [39]:
hist_df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                    'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']

In [40]:
hist_df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
0,1514764800000,13715.65000000,13715.65000000,13400.01000000,13529.01000000,443.35619900,1514768399999,5993909.83603800,5228,228.52192100,3090541.10576949,0
1,1514768400000,13528.99000000,13595.89000000,13155.38000000,13203.06000000,383.69700600,1514771999999,5154521.55513544,4534,180.84040300,2430449.49295672,0
2,1514772000000,13203.00000000,13418.43000000,13200.00000000,13330.18000000,429.06457200,1514775599999,5710192.01852959,4887,192.23793500,2558504.65764531,0
3,1514775600000,13330.26000000,13611.27000000,13290.00000000,13410.03000000,420.08703000,1514779199999,5657448.45730419,4789,137.91840700,1858041.25745743,0
4,1514779200000,13434.98000000,13623.29000000,13322.15000000,13601.01000000,340.80732900,1514782799999,4588046.99615741,4563,172.95763500,2328057.92021612,0


In [ ]:
hist_df['Open Time'] = pd.to_datetime(hist_df['Open Time']/1000, unit='s')

In [53]:
hist_df['Open Time'][0]

1514764800000

In [94]:
pd.to_datetime(hist_df['Open Time'][0], unit='ms')

Timestamp('2018-01-01 00:00:00')